In [1]:
%matplotlib inline
from parcels import Field, FieldSet, ParticleSet,Variable, JITParticle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc, animation
import xarray as xr

from IPython.display import Image
rc('animation', html='html5')

In [2]:
def tu_mix(particle,fieldset,time):
    bath= 4.5
    if particle.depth+0.1 > bath: #Only calculate gradient of diffusion for particles deeper than 0.6 otherwise OP will check for particles outside the domain and remove it.
        Kzdz = 10*(fieldset.Kz[time,particle.depth,particle.lat, particle.lon]-fieldset.Kz[time,particle.depth-0.1,particle.lat, particle.lon]) #forward difference 
    else: 
        Kzdz = 10*(fieldset.Kz[time,particle.depth+0.1,particle.lat, particle.lon]-fieldset.Kz[time,particle.depth,particle.lat, particle.lon]) #forward difference 
    dgrad = Kzdz * particle.dt 
    if particle.depth+0.5*dgrad > 0 and particle.depth+0.5*dgrad < bath:
        kk = fieldset.Kz[time,particle.depth+0.5*dgrad,particle.lat, particle.lon] #Vertical diffusivity SSC  #
    else:
        kk = fieldset.Kz[time, particle.depth,particle.lat, particle.lon] #Vertical diffusivity SSC  #
    Rr = ParcelsRandom.uniform(-1, 1)
    d_random = sqrt(3*2*kk*particle.dt) * Rr
    dzs = d_random +dgrad

def Buoyancy(particle, fieldset, time):
    """Calculating settling velocity using Komar cylinder Vs"""
#Check particle is in the water column   
    d = 2.16e-5 # particle diameter
    l = 4.5e-4 # particle length
    g = 9.8 #Gravity
    visc = 1e-3
    Ws= ((l/d)**-1.664)*0.079*((l**2)*g*(1350-1025))/(visc)
    dws = Ws*particle.dt

def displacement(particle, fieldset, time):
    if dzs +particle.depth + dws > bath: #randomly in the water column
        particle.depth = 2*bath - (dzs +particle.depth+dws)
    elif dzs +particle.depth +dws < 0:
        particle.depth = -(dzs +particle.depth+dws) #Well mixed boundary layer
    else:
        particle.depth += dzs + dws

In [3]:
Dat = pd.read_csv('ubcSSg3DwGridFields1hV19-05_9244_b1de_6bbd.csv')
depth=np.array(Dat.depth)[1:].astype('float32')
Kz_col=np.array(Dat.vert_eddy_diff)[1:].astype('float32')

In [16]:
Kz_col

array([0.        , 0.00659723, 0.00675717, 0.00937495, 0.0139404 ,
       0.01922989, 0.02459853, 0.03021166, 0.03556296, 0.04029927,
       0.04755556, 0.05717738, 0.069768  , 0.08511511, 0.10556598,
       0.13037497, 0.16046694, 0.19627848, 0.23991992, 0.2968998 ,
       0.36961082, 0.47269472, 0.61203265, 0.68819815, 0.4015745 ,
       0.14848088, 0.00562765, 0.        ], dtype=float32)

In [5]:
dim = 10
dimz = len(depth)
R = 0.5
L = 4.5
#Kz_col= np.repeat(0.03,dimz)
#Kz_col[-5:]=0
#depth = np.linspace(0,L,dimz, dtype=np.float32)
lon = np.linspace(0., R*2, dim, dtype=np.float32)

U = Field('U', np.zeros((dimz, dim), dtype=np.float32), lon=lon, depth=depth)
V = Field('V', np.zeros((dimz, dim), dtype=np.float32), lon=lon, depth=depth)
Kz_data = np.zeros((dimz, dim), dtype=np.float32)
for i in range(dim):
    Kz_data[:,i]=Kz_col
Kz = Field('Kz', Kz_data, grid=U.grid)

In [6]:
fieldset = FieldSet(U,V)
fieldset.add_field(Kz)

In [11]:
Vol = 2*R*depth[-1]
C0 = 1
n = int(C0*Vol * 1000)
lon_g = np.random.uniform(low=lon[0], high=lon[-1], size=(n,))
depth_g = np.random.uniform(low=depth[0], high=depth[-1], size=(n,))
lat_g = np.zeros(n,)

In [12]:
pset = ParticleSet(fieldset, pclass=JITParticle, lon=lon_g, depth=depth_g,lat = lat_g)
output_file = pset.ParticleFile(name='/home/jvalenti/MOAD/results/Outputmix.zarr', outputdt=50)
KE=[tu_mix,Buoyancy,displacement]
pset.execute(KE , runtime=5e3, dt=5, output_file=output_file)

  0%|          | 5.0/5000.0 [01:13<20:21:35, 14.67s/it]
INFO: Output files are stored in /home/jvalenti/MOAD/results/Outputmix.zarr.


FieldOutOfBoundSurfaceError: Field sampled out-of-bound at the surface, at (0.2032553106546402, 0.0, 74.18266296386719)

In [ ]:
dat = xr.load_dataset('/home/jvalenti/MOAD/results/Outputmix.zarr')

: 

In [ ]:
from tqdm import tqdm
import time
def anim2(file1,fps=1):  
    box=([0,1,1,0,0],[0,0,depth[-1],depth[-1],0])
    fig,axs=plt.subplots(1,3,figsize=(15,10))
    axs[0].scatter(dat.lon[:,0],dat.z[:,0],s=5)
    axs[2].plot(Kz_col,depth,c='r')
    axs[2].set_title('Vertical profile $K_z$ [$m^2/s$]')
    axs[2].invert_yaxis()
    axs[0].plot(box[0],box[1],c='grey',linewidth=3)
    axs[1].plot(box[0],box[1],c='grey',linewidth=3)
    axs[0].invert_yaxis()
    axs[0].set_title('Initial condition (t: 0s)')
    axs[1].invert_yaxis()
    axs[1].set_title("With $K_z$' correction (t: 5*$10^3$s)")
    axs[0].set_ylabel('Depth (m)')

    def update(frame):         
        global ss        
        for scat in ss:              
            scat.remove()            
        ss =[]                    
        ss.append(axs[1].scatter(file1.lon[:,frame],file1.z[:,frame],s=5,c ='tab:blue'))    
        time.sleep(3)
        print(f'{frame*100/file1.lon.shape[1]:.2f}% completed')           
        return ss
    return animation.FuncAnimation(fig, update, frames=np.arange(0,file1.lon.shape[1],fps))

: 

In [ ]:
ss =[]
ani = anim2(dat,fps=10)
f = r"/home/jvalenti/MOAD/animations/mix.gif" 
FFwriter = animation.FFMpegWriter()
ani.save(f, writer = FFwriter)

: 

In [ ]:
with open(f,'rb') as anim:
     display(Image(anim.read()))

: 

In [ ]:
def entropy(data, bins=100):
    """
        Calculate entropy of multiple varibles (discrete and continous).
        X is a 2d-array, each column is a variable.
    """
    hist = np.histogramdd(data, bins=bins)[0]
    prob = hist/len(data)
    prob[prob == 0] = 1
    log_prob = np.log2(prob)
    

    return -np.sum(np.multiply(prob, log_prob))

: 

In [ ]:
def no_nan(x,y):
    no_nan=[]
    no_nan1=[]
    for i,xi in enumerate(x):
        if np.isnan(xi) == False:
            no_nan.append(xi) 
            no_nan1.append(y[i]) 
    return no_nan,no_nan1

d0,l0=no_nan(dat.lat[:,0],dat.lon[:,0])
e0=entropy([d0,l0])

: 

In [ ]:
import sys, os

# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__

: 

In [ ]:
def run_n_test(n):
    e1,e2=[],[]
    for i in range(n):
        lon_g = np.random.uniform(low=lon[0], high=lon[-1], size=(10*dim,))
        depth_g = np.random.uniform(low=depth[0], high=depth[-1], size=(10*dim,))
        run_turb_test(lon_g,depth_g)
        dat = xr.load_dataset('/home/jvalenti/MOAD/results/Outputmix.zarr')
        dat2 = xr.load_dataset('/home/jvalenti/MOAD/results/Outputmix2.zarr')
        d0,l0=no_nan(dat.z[:,0],dat.lon[:,0])
        e0=entropy([d0,l0])
        d2,l2=no_nan(dat2.z[:,-1],dat2.lon[:,-1])
        e2.append(entropy([d2,l2])-e0)
        d1,l1=no_nan(dat.z[:,-1],dat.lon[:,-1])
        e1.append(entropy([d1,l1])-e0)
    m1=np.mean(e1)
    sd1=np.std(e1)/np.sqrt(n)
    m2=np.mean(e2)
    sd2=np.std(e2)/np.sqrt(n)
    return m1,sd1,m2,sd2

: 

In [ ]:
blockPrint()
m1,sd1,m2,sd2=run_n_test(10)
enablePrint()

CTEs = [m1, m2]
error = [sd1, sd2]

: 

In [ ]:
d1,l1=no_nan(dat.lat[:,-1],dat.lon[:,-1])
entropy([d1,l1])-e0

: 